In [56]:
import requests
import json
import boto3
import pandas as pd
import time
import os

In [2]:

ACCESS_KEY = ''
SECRET_KEY = ''
API_KEY = ''

In [3]:
# S3
REGION = 'ca-central-1'
BUCKET = 'quetzal-immense'

# api gateway
url = 'https://lud5uqi5j5.execute-api.ca-central-1.amazonaws.com/dev'
stateMachineArn = 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense'

In [4]:
headers = {
        'Accept': '*/*',
        'Content-Type':'application/json',
        'x-api-key' : API_KEY,
}

# Inputs (S3 structure)

am/inputs/road/road_times.csv <br>
pm/inputs/road/road_times.csv <br>
ip/inputs/road/road_times.csv <br>
op/inputs/road/road_times.csv <br>

scenario/inputs/road_skims.csv<br>
scenario/outputs/volumes.csv<br>
<br>
here scenario is up to you but should be provided in the api call
<br>
you may see other files (such as params.json) those are either used in the web interface are create by the model.

In [6]:
periods = ['am', 'pm', 'ip', 'op']

In [7]:
session = boto3.Session(
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY)

s3 = session.client('s3')

In [8]:
inputs_list=[]
for period in periods:
    prefix = period + '/inputs/'
    response = s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)
    for obj in response['Contents']:
        inputs_list.append(obj["Key"])

In [11]:
#inputs_list

<b>NOTE:</b> Constant inputs maybe stored into the model docker directly.<br>
zones, reference volume and reference skims are store into the docker and not on S3 here

At this moment, Only one period can be run per scenario. <br>
the solution is to run a scenario per period and have another scenario (running the demand model) that will used data from those period and return the outputs <br>
<br>
In this example, this mean that we need to add our inputs under am/ and pm/ and run a third model (named example here). This model will run both am and pm in parallel and run the demand model. the results will be under example/outputs/
<br>
<br>
The last scenario (example) can have any name, the period one cannot at this moment. this mean that you can only run one simulation at the time as they all use the same s3 folder.

# Run

In [37]:
# important
# this is where the road_skims.csv are: (example/inputs/road_skims.csv)
# this will be the outputs s3 path (example/outputs/volumes.csv)
# can be any name (except am, pm, op or ip)
scenario = 'example'

In [38]:
response = s3.list_objects_v2(Bucket=BUCKET, Prefix=scenario)
for obj in response['Contents']:
    print(obj["Key"])

example/inputs/params.json
example/inputs/road_skims.csv
example/outputs/volumes.csv


In [44]:
inputs = { 
        'scenario_path_S3': scenario, # necessary
        'choice': 'orchestrator', # necessary: orchestrator | test
        'metadata' : {"user_email": "test@test.com"}, # necessary. it is used in the WebApp to show last modified
        'params':{'general': {'scenario_path_S3': scenario}} # used to save outputs on immense bucket.
}
inputs = json.dumps(inputs)


In [45]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
data

'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense", "input": "{\\"scenario_path_S3\\": \\"example\\", \\"choice\\": \\"orchestrator\\", \\"metadata\\": {\\"user_email\\": \\"test@test.com\\"}}"}'

POST start quetzal model execution

In [46]:
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

execution arn is returned in the post response and used to get status or end the simulation

In [47]:
executionArn = resp.json()['executionArn']
executionArn

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:1665da23-0be9-41da-8ce2-0f47e18b9de8'

# Polling Simulation Status

RUNNING | SUCCEEDED | FAILED | TIMED_OUT | ABORTED 

In [48]:
def polling(executionArn):
    data = { "executionArn": executionArn }
    data=json.dumps(data, ensure_ascii=False).encode('utf-8')
    resp =  requests.post(url+'/describe',data=data,headers=headers)
    resp.ok

    status =  'RUNNING'
    while True:
        resp =  requests.post(url+'/describe',data=data,headers=headers)
        status = resp.json()['status']
        print(status)
        if status != 'RUNNING':
            break
        time.sleep(10)
polling(executionArn)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
SUCCEEDED


You could get the step function definition and poll its current Step if needed. <br>
However, it is a bit difficult to decrypt and not really relevent for this project in my opinion


# Outputs

In [58]:
prefix = os.path.join(scenario,'outputs')
ls = []
response = s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)
for obj in response['Contents']:
    file = obj["Key"]
    ls.append(file)
    print(file)

example/outputs/volumes.csv


In [59]:
from io import BytesIO
data = s3.get_object(Bucket=BUCKET, Key=ls[0])
contents = data['Body'].read()
with BytesIO(contents) as bio:
    df = pd.read_csv(bio)

In [60]:
df

,index,origin,destination,time_period,vehicle_class,purpose,volume
0,0,1,80,AM,C_other,HBW,1.120000
1,1,1,91,AM,C_other,HBW,1.120000
2,2,1,101,AM,C_other,HBW,1.120000
3,3,1,103,AM,C_other,HBW,14.560000
4,4,1,172,AM,C_other,HBW,2.239992
...,...,...,...,...,...,...,...
726482,726482,701,694,OP,RPAX,ALL,0.162973
726483,726483,701,696,OP,RPAX,ALL,0.166852
726484,726484,701,697,OP,RPAX,ALL,0.164810
726485,726485,701,698,OP,RPAX,ALL,0.916444


# Stop simulation

In [297]:
data = { "executionArn": executionArn }
data = json.dumps(data)
resp =  requests.post(url+'/abort',data=data,headers=headers)
resp.ok


True

In [298]:
resp.json()

{'stopDate': 1707756039.138}

In [299]:
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [300]:
resp.json()['status'] # will return ABORTED. if not. the simulation was done are had an error.

'ABORTED'

# test (faster)

<b>choice = test</b> is a simple endpoint to test. it run in ~5secs and will create a  file outputs/test.txt with it's current time (timestamp in seconds)

In [33]:
scenario = 'test/'
inputs = { 
        'scenario_path_S3': scenario, # necessary anything
        'choice': 'test', # necessary: orchestrator | test
        'metadata' : {"user_email": "test@test.com"} # necessary. it is used in the WebApp to show last modified
}
inputs = json.dumps(inputs)

In [34]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

In [35]:
executionArn = resp.json()['executionArn']
polling(executionArn)

RUNNING
SUCCEEDED


In [36]:
data = s3.get_object(Bucket=BUCKET, Key=scenario + 'outputs/test.txt')
contents = data['Body'].read()
contents

b'test passed! \ntime: 1709846103'